In [2]:
import pandas as pd
import ast


filename = '../healsl_rd1to2_rapid_gpt3_v2b_2024_03_15/healsl_rd1_rapid_gpt3_v2b_first_ICD.csv'
df = pd.read_csv(filename)

df.output_probs = df.output_probs.apply(ast.literal_eval)

In [3]:
if "gpt3" in filename:
    model_name = "GPT3"
elif "gpt4"in filename:
    model_name = "GPT4"

if "first_ICD" in filename:
    sort_scheme = "First ICD"
    test_sort = False
elif "sorted_ICD"  in filename:
    sort_scheme = "Highest Prob"
    test_sort = True
    
if "rd1" in filename:
    round_num = "Round 1"
elif "rd2" in filename:
    round_num = "Round 2"
    
if "sample" in filename:
    sample = "Sample"
else:
    sample = "Full"
    

In [4]:
def prob_ICD(probs, sort=False):
    if probs == []:
        return []
    temp = pd.DataFrame(probs)    
    
    if sort:
        temp = temp.sort_values(by='icd_linprob_mean', ascending=False)
    
    # display(temp)
    return temp.icd.to_list()

In [5]:
df['cause_list'] = df[['cause1_icd10', 'cause2_icd10', 'cause3_icd10', 'cause4_icd10', 'cause5_icd10']].apply(lambda x: x.dropna().tolist(), axis=1)
df['sanity'] = df.output_probs.apply(prob_ICD, sort=test_sort)


In [6]:
print(f"Model: {model_name}, Sort: {sort_scheme}, {round_num}")
print(f"Shape: {df.shape}")

error_flag = False
for idx, a in df[['cause_list', 'sanity']].iterrows():
    if a.cause_list[:5] != a.sanity[:5]:
        print(idx)
        print(a.cause_list)
        print(a.sanity)
        error_flag = True
        
if error_flag:
    print("Mismatch found!")
else:
    print("No mismatch found!")
    

Model: GPT3, Sort: First ICD, Round 1
Shape: (8570, 25)
No mismatch found!


In [7]:
df.output_created

0       2024-03-09T04:02:34.613342-05:00
1       2024-03-09T04:02:35.114541-05:00
2       2024-03-09T04:02:35.921537-05:00
3       2024-03-09T04:02:36.160631-05:00
4       2024-03-09T04:02:36.554688-05:00
                      ...               
8565    2024-03-10T04:38:10.530308-04:00
8566    2024-03-10T04:38:11.751086-04:00
8567    2024-03-10T04:38:12.048447-04:00
8568    2024-03-10T04:38:12.551756-04:00
8569    2024-03-10T04:38:12.852355-04:00
Name: output_created, Length: 8570, dtype: object